In [1]:
from os.path import join

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LassoCV, ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

#### I/O

In [2]:
datasets_paths = {
    "Newcastle": "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/features_source_ind-bands",
    "Izmir": "/Volumes/MMIS-Saraiv/Datasets/Izmir/EC/features_source_ind-bands",
    "Istambul": "/Volumes/MMIS-Saraiv/Datasets/Istambul/features_source_ind-bands",
}

In [3]:
datasets_metadata_paths = {
    "Izmir": "/Volumes/MMIS-Saraiv/Datasets/Izmir/metadata.csv",
    "Istambul": "/Volumes/MMIS-Saraiv/Datasets/Istambul/metadata.csv",
    "Newcastle": "/Volumes/MMIS-Saraiv/Datasets/Newcastle/metadata.csv",
}

In [4]:
plots_path = "/Users/saraiva/Desktop/Doktorand/Combat Project"

#### Functions

In [5]:
def get_subject_code(identifier: str) -> int:
    return int(identifier.split("PARTICIPANT")[1])

In [6]:
def dataset_norm(df: pd.DataFrame, method:str):
    if method == 'z-score':  # Z-score
        return df.apply(zscore)
    elif method == 'min-max':
        return (df - df.min()) / (df.max() - df.min())
    elif method == 'log10':
        return df.applymap(lambda x: np.log10(x) if x > 0 else 0)

In [7]:
def inter_dataset_norm(df: pd.DataFrame):
    pass

In [8]:
def read_dataset(path: str, label:str) -> pd.DataFrame:
    df = pd.read_csv(join(path, "all_features.csv"), index_col=0)
    df.index = [(f"{label}-{get_subject_code(s)}" if isinstance(s, str) else f"{label}-{s}") for s in df.index]
    return df

In [9]:
def read_metadata(path: str, label: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col=0)
    df.index = [(f"{label}-{get_subject_code(s)}" if isinstance(s, str) else f"{label}-{s}")  for s in df.index]
    return df

In [10]:
def neuro_harmonize(_X: pd.DataFrame, _metadata: pd.DataFrame) -> pd.DataFrame:
    from neuroHarmonize import harmonizationLearn
    assert _X.index.tolist() == _metadata.index.tolist()
    data = _X.to_numpy(dtype=np.float32)
    covariates:pd.DataFrame = _metadata.copy()
    # keep only the age and gender columns
    covariates = covariates[['AGE', 'GENDER']]
    # binarize gender: M=1, F=0
    covariates['GENDER'].replace('M', 1, inplace=True)
    covariates['GENDER'].replace('F', 0, inplace=True)
    # add column 'SITE' with the dataset name
    covariates['SITE'] = [s.split('-')[0] for s in covariates.index]
    # run
    combat_model, harmonized_data = harmonizationLearn(data, covariates)
    return pd.DataFrame(harmonized_data, index=_X.index, columns=_X.columns)

#### Read

In [11]:
# Read datasets
datasets = {}
for dataset_name, path in datasets_paths.items():
    dataset = read_dataset(path, label=dataset_name)
    datasets[dataset_name] = dataset 
#datasets

In [12]:
# Read metadata
datasets_metadata = {}
for dataset_name, path in datasets_metadata_paths.items():
    dataset = read_metadata(path, label=dataset_name)
    datasets_metadata[dataset_name] = dataset 
#datasets_metadata

In [13]:
# Join all datasets and metadata
X = pd.concat(datasets.values())
all_metadata = pd.concat(datasets_metadata.values())
all_metadata = all_metadata.loc[X.index]  # keep only the metadata of the subjects in X
assert X.shape[0] == all_metadata.shape[0]

In [14]:
# Print class frequency
all_metadata['DIAGNOSIS'].value_counts()

In [15]:
class_freq = all_metadata['DIAGNOSIS'].value_counts(normalize=True)
class_freq

In [16]:
# Make integer targets (diagnosis)
targets = all_metadata['DIAGNOSIS'].astype('category').cat.codes

#### Inter-Normalisation & Harmonisation

In [17]:
selected_features = ['Alpha1_Frontal', 'Alpha1_Central', 'Alpha1_Temporal',
       'Theta_Occipital', 'Alpha3_Parietal', 'Alpha3_Occipital',
       'Alpha3_Temporal', 'Alpha3_Limbic', 'Beta1_Central', 'Beta1_Occipital']
X = X[selected_features]

In [18]:
# Inter-dataset normalization
#X = dataset_norm(X, method='z-score')
X = dataset_norm(X, method='log10')

In [19]:
# NeuroHarmonize
X = neuro_harmonize(X, all_metadata)

In [20]:
X = dataset_norm(X, method='z-score')

In [21]:
X

#### Feature Selection

In [22]:
# Model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
#model = SVC(kernel='linear', C=1, degree=3)
model

In [23]:
# RFE
rfe = RFE(estimator=model, n_features_to_select=10, step=1)
rfe.fit(X, targets)
selected_features = X.columns[rfe.support_]
selected_features

In [24]:
# Selected biomarkers [Hardcoded]
selected_features = ['Alpha1_Occipital', 'Alpha1_Temporal', 'Theta_Temporal', 'Theta_Limbic',
       'Alpha3_Parietal', 'Alpha3_Occipital', 'Alpha3_Limbic',
       'Beta1_Parietal', 'Beta1_Occipital', 'Beta1_Limbic']

In [25]:
selected_features = ['Alpha1_Frontal', 'Alpha1_Central', 'Alpha1_Temporal',
       'Theta_Occipital', 'Alpha3_Parietal', 'Alpha3_Occipital',
       'Alpha3_Temporal', 'Alpha3_Limbic', 'Beta1_Central', 'Beta1_Occipital']

In [26]:
# Keep relevant features only
X = X[selected_features]
X

#### Analytics

In [27]:
# Mean and Std of all features
# AD
ad = X[targets == 1]
mean, std = ad.mean(), ad.std()
print("AD:")
for i, (m, s) in enumerate(zip(mean, std)):
    print(f"{selected_features[i]}: {m:.2f} +- {s:.2f}")
# HC
hc = X[targets == 0]
mean, std = hc.mean(), hc.std()
print("HC:")
for i, (m, s) in enumerate(zip(mean, std)):
    print(f"{selected_features[i]}: {m:.2f} +- {s:.2f}")

#### Classification

In [28]:
# Model
model = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=0)
# Linear discriminant analysis
#model = ElasticNet(alpha=0.1, l1_ratio=0.1)

In [29]:
# Leave-One-Out CV
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

loo = LeaveOneOut()
accuracies = []
for i, (train_index, test_index) in enumerate(loo.split(X)):
    print(f"Fold {i+1}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = targets.iloc[train_index], targets.iloc[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    accuracies.append(accuracy_score(y_test, y_pred))
    
mean_accuracy = sum(accuracies) / len(accuracies)
std_accuracy = sum([(a - mean_accuracy)**2 for a in accuracies]) / len(accuracies)
print(mean_accuracy, '+-', std_accuracy)